In [2]:
import pandas as pd
import numpy as np
from googletrans import Translator
from joblib import Parallel, delayed
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
%matplotlib inline

from google.cloud import translate

In [2]:
test = pd.read_csv("input/test.csv")
test.head()

,id,content,lang
0,0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru
2,2,"Quindi tu sei uno di quelli conservativi , ...",it
3,3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
4,4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr


In [19]:
test = pd.read_csv("input/test.csv")

In [68]:
# valid = pd.read_csv("input/validation.csv")
train = pd.read_csv("input/jigsaw-toxic-comment-train.csv")

In [239]:
en1 = pd.read_csv("test_fold/validation_en_1.csv")


In [262]:
test_en = pd.read_csv("test_en.csv")

In [259]:
valid_en = pd.read_csv("validation_en.csv")

In [260]:
valid_en

,id,comment_text,lang,toxic,content_en
0,0,Este usuario ni siquiera llega al rango de ...,es,0,This user does not even reach the rank of here...
1,1,Il testo di questa voce pare esser scopiazzato...,it,0,The text of this entry seems to be covered dir...
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,es,1,OK. I only expose my past. All past time was b...
3,3,Bu maddenin alt başlığı olarak uluslararası i...,tr,0,"As a subtitle of this article, I am hesitant a..."
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,tr,0,I guess you will take the Portekizi as an exam...
...,...,...,...,...,...
7995,7995,Il fatto è che la pagina dei personaggi minor...,it,0,The fact is that we understand the page of the...
7996,7996,El imbesil ete dela luna no se entera ni ostia...,es,1,The imbesil ete of the moon does not know or o...
7997,7997,olum sız manyakmısınz siz adam sıze sanal yıld...,tr,1,"dear, you are crazy, you don't know if the man..."
7998,7998,El mapa del reinado de Alhaken esta ligerament...,es,0,The map of the reign of Alhaken is slightly ex...


In [195]:
en_val = pd.read_csv("test_fold/validation_en.csv")

In [189]:
len(en_val)-1

7999

In [13]:
48160/301

160.0

In [134]:
chunk=10
test.iloc[chunk*301 : (chunk+1)*301, :]

,id,content,lang
301,301,20px Bem-vindo(a) à Wikipédia. Todas as pessoa...,pt
302,302,Şehrin en pahalı yerlerinde Hırvatlar oturur v...,tr
303,303,60px|left A página que criou com o nome :Bi...,pt
304,304,"Duas coisas. Primeiro, você colocou no título ...",pt
305,305,", bizimkisi millî olsa da, bu ülkelerdeki biri...",tr
...,...,...,...
597,597,Mi spiace darVi la brutta notizia. Ma non avre...,it
598,598,"Un altra cosa, e poi basta altrimenti mi ammaz...",it
599,599,Убрана Тема - Путин и бляди ? У нас есть осн...,ru
600,600,"dato che dovevo testare una cosa su en.wiki, t...",it


In [273]:
test_en[test_en.index == 2725].content.values

array(['СВ*НЬИ!!!!!!!!!! Вы удалили мои комментарии, и ЛОЖНО объясняете НАСЛЕДИЕ Мишеля! ЧТО ТЫ ТУТ ПИШЕШЬ, когда сам НИ БУКВ НИ ЧИСЕЛ не разумеешь?! Глупые перед Глупцами толкуют! Что РЕЗУЛЬТАТ - как не ГЛУПОСТЬ!!! Стоит ли теперь МНЕ перед СВИНЬ*МИ распинаться и ДАВАТЬ вам ИСТИНУ? ...СОТРЁТЕ! СОЛГЁТЕ! ИЗВРАТИТЕ! ДАМ В ПОСЛЕДНИЙ РАЗ... но не ШЛЮХ*М и ГЛУПЦАМ... лишь тем, кто идёт  ПУТЁМ , коим заповедовал СЫН БОЖИЙ!!! - Смотри на герб не как на ГЕРБ. Возьми  ИМЯ ЕДИНОГО  и НАЛОЖИ на ГЕРБ. Узнаешь кто БЫЛ и ЕСТЬ Нострадамус. Соединение ГЛАЗ и ЦЕНТРА колёс - вот твои ориентиры. Найдя  ИМЯ ЕДИНОГО  - подели на 2, - ТАМ НАСЛЕДИЕ РУССКОГО НАРОДА ВОВЕК! Господь с вами. Мир вам. Аминь. ДУНАДАН'],
      dtype=object)

In [274]:
test_en[test_en.index == 2725].content_en.values

array(["SV * NI !!!!!!!!!! You have deleted my comments, and FALSE explain Michel's HERITAGE! WHAT DO YOU WRITE IN WHEN YOU DO NOT UNDERSTAND NUMBER OF LETTERS ?! The fools are interpreting before the fools! What a RESULT - if not STUPID !!! Is it worth ME now before PIG * MI to crucify and GIVE you the TRUTH? ... REMOVE! LIE! Pervert! LADY FOR THE LAST TIME ... but not a WHORE * M and FUELS ... only to those who follow the WAY that the SON OF GOD commanded !!! “Look at the coat of arms not like the COAT OF ARMS. Take ONE NAME and COAT OF ARMS. You find out who WAS and IS Nostradamus. The combination of the EYE and the CENTER of wheels is your guideline. Finding the ONE NAME - divided by 2, - THERE IS THE HERITAGE OF THE RUSSIAN PEOPLE FOREVER! The Lord is with you. Peace to you. Amen. DUNADAN"],
      dtype=object)

In [268]:
import re
# print(re.findall("\w\S*@", line))
dct_ru={}
ru_asterisk = test_en[test_en['content'].str.contains("\*+")].query("lang=='ru'")
for idx, row in zip(ru_asterisk.index, ru_asterisk.content):
     dct_ru[idx] = [word for word in row.split(' ') if '*' in word]

In [269]:
dct_ru

{630: ['п**ды'],
 860: ['(*в', 'убегает*)'],
 891: ['ё*нулись,'],
 1096: ['пи.*да', 'пи*+да-/ра+*сам...'],
 1150: ['ж*ды,', 'ч*рки,', 'чернож*пые,'],
 1518: ['х**м!'],
 1545: ['е****т'],
 2087: ['ху*ня'],
 2151: ['пи*-да-*рас-*а.',
  '-1*до',
  '*-р',
  '-*ство',
  'пи-1*д-*о-*р-*1ов',
  'пи*',
  '*р',
  '-*а',
  '*1и',
  '-1*до-',
  '1*р-*'],
 2246: ['пед*викии', 'лурко*ба!'],
 2506: [',лурко*б', 'ср*ное', 'луркомо*ье', 'лги,пиз*и'],
 2613: ['контент;*Адекватная'],
 2725: ['СВ*НЬИ!!!!!!!!!!', 'СВИНЬ*МИ', 'ШЛЮХ*М'],
 3050: ['ду**ктакое'],
 3212: ['(*'],
 3339: ['е*****?'],
 3673: ['Уголовного**'],
 4327: ['зае**л!', 'за**ал', 'б**дь', 'пога**я', 'еб*ом!'],
 7169: ['Говн*'],
 7330: ['92.113.156.***,', '86.110.156.***'],
 8561: ['д*ст*л.', 'п*р*дке', 'г*л*в*й?', 'п*р*шься', 'п*р*шь?'],
 8673: ['чернож***е'],
 8836: ['ж***'],
 9828: ['**Maxim'],
 10098: ['-=|*Altes*|=-'],
 10596: ['вагине,бл*дь,или', 'место,бл*дь!'],
 11826: ['Letzte*Spieler'],
 12124: ['-х**', '-п****', '-б**дь', '-б**ть

{15: ['СВ*НЬИ!!!!!!!!!!', 'СВИНЬ*МИ', 'ШЛЮХ*М'],
 340: ['ду**ктакое'],
 502: ['(*'],
 629: ['е*****?'],
 963: ['Уголовного**'],
 1617: ['зае**л!', 'за**ал', 'б**дь', 'пога**я', 'еб*ом!'],
 4459: ['Говн*'],
 4620: ['92.113.156.***,', '86.110.156.***'],
 5851: ['д*ст*л.', 'п*р*дке', 'г*л*в*й?', 'п*р*шься', 'п*р*шь?'],
 5963: ['чернож***е'],
 6126: ['ж***'],
 7118: ['**Maxim'],
 7388: ['-=|*Altes*|=-'],
 7886: ['вагине,бл*дь,или', 'место,бл*дь!'],
 9116: ['Letzte*Spieler'],
 9414: ['-х**', '-п****', '-б**дь', '-б**ть'],
 11335: ['Letzte*Spieler,'],
 11929: ['*'],
 12709: ['Цензоры-викип*ры'],
 12888: ['зас*ть.'],
 13450: ['х*есосова'],
 13944: ['идея*)'],
 14302: ['пи***ж'],
 14654: ['***'],
 15082: ['2*', '*', '2*'],
 15197: ['*'],
 15786: ['_p*dik'],
 16354: ['вики*****оф.'],
 17016: ['ё***ый'],
 17494: ['*'],
 18443: ['-=|*', '*|=-'],
 18512: ['*'],
 18767: ['к*.'],
 19353: ['*лядь', 'д*бил', 'с*ка', 'е*аный!!!'],
 19603: ['ни*уя.'],
 19610: ['/*'],
 19898: ['/*', '*/'],
 20111: ['*неч

In [264]:
test_en[test_en.lang == 'ru']

,id,content,lang,content_en
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru,"It is possible, but so far I do not see the ne..."
18,18,", под вашу ответственность. NB: ЭСБЕ, значит, ...",ru,", under your responsibility. NB: ESBE means sh..."
20,20,"Ошибаетесь Вы. Во-первых, текст должен быть п...",ru,"You are mistaken. Firstly, the text should be ..."
26,26,"Отличная работа, спасибо. Я чуть доработал - п...",ru,Excellent work. Thank you. I finished it a bit...
28,28,левакофашист и путинский лизун ??? Как вообще ...,ru,left-wing fascist and Putin’s lizun ??? How di...
...,...,...,...,...
63790,63790,Источник не авторитетный. Написано под редакци...,ru,The source is not authoritative. It is written...
63794,63794,См. Великий марш возвращения → Столкновения на...,ru,See Great March of Return → Clashes on the Isr...
63799,63799,Предлагаю добавить вот ещё что про коммерческу...,ru,I propose to add one more thing about a commer...
63801,63801,... получил сталинскую премию за музыку к филь...,ru,... received the Stalin Prize for film music. ...


In [267]:
valid_en

,id,comment_text,lang,toxic,content_en
0,0,Este usuario ni siquiera llega al rango de ...,es,0,This user does not even reach the rank of here...
1,1,Il testo di questa voce pare esser scopiazzato...,it,0,The text of this entry seems to be covered dir...
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,es,1,OK. I only expose my past. All past time was b...
3,3,Bu maddenin alt başlığı olarak uluslararası i...,tr,0,"As a subtitle of this article, I am hesitant a..."
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,tr,0,I guess you will take the Portekizi as an exam...
...,...,...,...,...,...
7995,7995,Il fatto è che la pagina dei personaggi minor...,it,0,The fact is that we understand the page of the...
7996,7996,El imbesil ete dela luna no se entera ni ostia...,es,1,The imbesil ete of the moon does not know or o...
7997,7997,olum sız manyakmısınz siz adam sıze sanal yıld...,tr,1,"dear, you are crazy, you don't know if the man..."
7998,7998,El mapa del reinado de Alhaken esta ligerament...,es,0,The map of the reign of Alhaken is slightly ex...


In [50]:
np.mean(cnt_words), np.max(cnt_words), np.median(cnt_words)

(63.34225537516455, 460, 48.0)

In [71]:
!nordvpn connect

Connecting to Moldova #11 (md11.nordvpn.com)
The active UFW firewall on your system prevents us from setting up our firewall properly. We have disabled UFW for the duration of your VPN connection and enabled our firewall to ensure your online security. Your custom UFW rules are imported to our firewall ruleset.
You are connected to Moldova #11 (md11.nordvpn.com)!


In [70]:
!nordvpn set autoconnect on

Auto-connect is already set to 'enabled'.
